In [ ]:
%%file 1a.py

#Importing libraries
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD

#Get rank of the communicator
rank = comm.Get_rank()

#Size of the process
size = comm.Get_size()

#Variable Initialization
N = int(16)
sum_xy = []
executiontime = 0

#Master Process
if rank != 0:
    start = MPI.Wtime()    
    a = comm.recv(source = 0)
    b = comm.recv(source = 0)
    comm.send(a+b, dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0,tag=2)
    print("Time taken by worker ",rank,"is: ", end - start)
    
#Worker Process    
else:
    start = MPI.Wtime()
    x = np.random.randint(100,size = N)
    y = np.random.randint(100,size = N)
    
    #Splitting the dataset depending on the size of the workers
    split_x = np.array_split(x,size)
    split_y = np.array_split(y,size)
    
    if rank == 0:
        #Master's work
        sum_xy.extend(list(split_x[rank]+split_y[rank]))
        
    for worker in range(1,size):
        #Pointopoint communication to worker
        comm.send(split_x[worker], dest = worker)
        comm.send(split_y[worker], dest = worker)
        sum_xy.extend(list(comm.recv(source = worker,tag=1)))
        executiontime+=comm.recv(source=worker,tag=2)

    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker ",rank,"is: ", end - start)
    print("Total executed time: ", np.sum(executiontime))
    print("\nX:",x)
    print("Y:",y)
    print("Sum:",sum_xy)

In [ ]:
!mpiexec -n 1 python 1a.py

In [ ]:
!mpiexec -n 2 python 1a.py

In [ ]:
%%file 1b.py

#Importing libraries
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD
#Get rank of the worker
rank = comm.Get_rank()
#Get size of the process
size = comm.Get_size()

#Initialize variable
N = int(1e4)
average = None
average_x = []
executiontime = 0

#Master Process
if rank != 0:
    start = MPI.Wtime()    
    a = comm.recv(source = 0)
    comm.send(np.sum(a)/len(a), dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0,tag=2)
    print("Time taken by worker ",rank,"is: ", end - start)

#Worker Process
else:
    start = MPI.Wtime()
    x = np.random.randint(100,size=N)
    
    #Split data based on number of workers
    split_x = np.array_split(x,size)
    
    #Master's work
    if rank == 0:
        average_x.append(np.sum(split_x[rank])/len(split_x[rank]))
        
    #Worker's part
    for worker in range(1,size):
        comm.send(split_x[worker], dest = worker)
        average_x.append(comm.recv(source = worker,tag=1))
        executiontime+=comm.recv(source=worker,tag=2)
    
    #Average of the vector
    average = np.average(average_x)
    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker ",rank,"is: ", end - start)
    print("Total executed time: ", np.sum(executiontime))
#     print("\nX:",x)
#     print("Average:",average)

In [ ]:
!mpiexec -n 1 python 1b.py

In [ ]:
!mpiexec -n 2 python 1b.py

In [ ]:
%%file 2VecMult.py

#Import Library
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = int(1e4)
VecMult = []
executiontime = 0

if rank != 0:
    start = MPI.Wtime()    
    A = comm.recv(source = 0)
    b = comm.recv(source = 0)
    comm.send(np.matmul(A,b), dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0,tag=2)
    print("Time taken by worker ",rank,"is: ", end - start)
    
else:
    start = MPI.Wtime()
    A = np.random.randint(100,size=(N,N))
    b = np.random.randint(100,size=(N,1))
    split_A= np.array_split(A,size)
    
    if rank == 0:
        out = np.matmul(split_A[rank],b)
        VecMult.extend(out.flatten().tolist())
        
    for worker in range(1,size):
        comm.send(split_A[worker], dest = worker)
        comm.send(b,dest=worker)
        out = comm.recv(source = worker,tag=1)
        VecMult.extend(out.flatten().tolist())
        executiontime+=comm.recv(source=worker,tag=2)
    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker ",rank,"is: ", end - start)
    print("Total executed time: ", np.sum(executiontime))
#     print("\nA:",A)
#     print("b:",b)
#     print("Vector Multiplication:",VecMult)
    print(len(VecMult))

In [ ]:
!mpiexec -n 1 python 2VecMult.py

In [ ]:
!mpiexec -n 2 python 2VecMult.py

In [ ]:
%%file sam.py

#Import Library
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

#Initialize Matrices
A = None
B = None
N = int(1e2)
C = np.zeros((N,N))


if rank != 0:
    B = np.empty((N,N))
else:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)

#Receiving variable
Arecv = np.empty((int(N/size),N))
start = MPI.Wtime()

#Separate data to all workers
comm.Scatter(A,Arecv,root=0)

#Send a copy of the vector to all workers
comm.Bcast(B,root=0)

#Gather the result from all workers
comm.Gather(np.matmul(Arecv,B),C,root=0)

end = MPI.Wtime()
print("Time taken by worker:",rank,"is: ",end-start)

In [ ]:
!mpiexec -n 1 python sam.py

In [ ]:
!mpiexec -n 2 python sam.py